In [1]:
import sys
!jupyter labextension install @jupyter-widgets/jupyterlab-manager jupyter-threejs
!jupyter nbextension install --py --symlink --sys-prefix pythreejs
!jupyter nbextension enable --py --sys-prefix pythreejs
#!{sys.executable} -m pip install pythreejs


(Deprecated) Installing extensions with the jupyter labextension install command is now deprecated and will be removed in a future major version of JupyterLab.

Users should manage prebuilt extensions with package managers like pip and conda, and extension authors are encouraged to distribute their extensions as prebuilt packages 
/home/runner/workspace/.pythonlibs/lib/python3.11/site-packages/jupyterlab/debuglog.py:54: UserWarning: An error occurred.
  warnings.warn("An error occurred.")
/home/runner/workspace/.pythonlibs/lib/python3.11/site-packages/jupyterlab/debuglog.py:55: UserWarning: ValueError: Please install Node.js and npm before continuing installation. You may be able to install Node.js from your package manager, from conda, or directly from the Node.js website (https://nodejs.org).
  warnings.warn(msg[-1].strip())
/home/runner/workspace/.pythonlibs/lib/python3.11/site-packages/jupyterlab/debuglog.py:56: UserWarning: See the log file for details: /tmp/jupyterlab-debug-9z8l3

In [13]:
# -*- coding: utf-8 -*-
"""golden spiral

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1Ge2j9ls2mRCKHBBLcdP5YlZUPbWngI2F
"""

import numpy as np
from scipy.optimize import root
import svgwrite
from IPython.display import display, SVG, clear_output
import ipywidgets as widgets
from typing import List, Tuple, Optional, Set, Dict
import random
import math
import itertools
from matplotlib.path import Path as MplPath

# ============================================
# Base Geometry and Drawing Classes
# ============================================

class Shape:
    """Base class for geometric shapes."""
    def __init__(self, visible: bool = True):
        """
        Initializes a Shape object.

        Args:
            visible: Whether the shape should be visible in the drawing.
        """
        self.visible = visible

    def to_svg(self, dwg: svgwrite.Drawing):
        """
        Abstract method to render the shape to an SVG element.

        Args:
            dwg: The svgwrite Drawing object.

        Raises:
            NotImplementedError: This method must be implemented by subclasses.
        """
        raise NotImplementedError

def lines_in_polygon(polygon, line_spacing=5, angle=0, color="#000000", stroke_width=0.5):
    """Fill polygon with parallel lines at a given angle, clipped to polygon bounds.
    
    Args:
        polygon: List of complex numbers or Nx2 numpy array representing polygon vertices
        line_spacing: Distance between parallel lines
        angle: Angle of lines in degrees (0 = horizontal)
        color: Color of the lines
        stroke_width: Width of the lines
    
    Returns:
        List of SVG line elements as svgwrite objects
    """
    # Convert complex points to numpy array if needed
    if polygon and isinstance(polygon[0], complex):
        polygon = np.array([(p.real, p.imag) for p in polygon])
    else:
        polygon = np.array(polygon)
    
    if polygon.shape[0] < 3:
        return []
    
    # Calculate bounding box
    min_x, min_y = polygon[:, 0].min(), polygon[:, 1].min()
    max_x, max_y = polygon[:, 0].max(), polygon[:, 1].max()
    bbox_diag = math.hypot(max_x - min_x, max_y - min_y)
    if bbox_diag == 0:
        return []
    
    centroid = polygon.mean(axis=0)
    theta = math.radians(angle)
    dx, dy = math.cos(theta), math.sin(theta)
    # Perpendicular direction for shifting parallel lines
    px, py = -dy, dx
    
    # Number of lines needed to cover bounding box
    num_lines = int(bbox_diag / max(1e-6, line_spacing)) + 3
    
    path = MplPath(polygon)
    line_elements = []
    
    # Ensure we sample at least a few points per candidate line
    sample_length = int(bbox_diag * 2)
    sample_length = max(sample_length, 40)
    
    for i in range(-num_lines, num_lines + 1):
        shift_x = px * i * line_spacing
        shift_y = py * i * line_spacing
        line_start = np.array([centroid[0] - dx * sample_length + shift_x,
                               centroid[1] - dy * sample_length + shift_y])
        line_end = np.array([centroid[0] + dx * sample_length + shift_x,
                             centroid[1] + dy * sample_length + shift_y])
        
        xs = np.linspace(line_start[0], line_end[0], sample_length)
        ys = np.linspace(line_start[1], line_end[1], sample_length)
        inside = [path.contains_point((x, y)) for x, y in zip(xs, ys)]
        
        start_idx = None
        for j, inside_flag in enumerate(inside):
            if inside_flag and start_idx is None:
                start_idx = j
            elif not inside_flag and start_idx is not None:
                x1, y1 = xs[start_idx], ys[start_idx]
                x2, y2 = xs[j - 1], ys[j - 1]
                line_elements.append(((x1, y1), (x2, y2)))
                start_idx = None
        # If a segment continues to the end
        if start_idx is not None:
            x1, y1 = xs[start_idx], ys[start_idx]
            x2, y2 = xs[-1], ys[-1]
            line_elements.append(((x1, y1), (x2, y2)))
    
    return line_elements

class DrawingContext:
    """
    Handles SVG drawing and coordinate normalization.

    Manages the SVG drawing object and scales geometric elements to fit within the viewbox.
    """
    def __init__(self, size: int = 800):
        """
        Initializes a DrawingContext.

        Args:
            size: The size of the square drawing area in pixels.
        """
        self.size = size
        self.dwg = svgwrite.Drawing(size=(size, size))
        self.scale_factor = 1.0

    def set_normalization_scale(self, elements: List['CircleElement']):
        """
        Calculates and sets the scale factor to fit elements into the viewbox.

        The scale factor is determined by the maximum extent of the circles (center + radius).

        Args:
            elements: A list of CircleElement objects to consider for scaling.
        """
        if not elements:
            self.scale_factor = 1.0
            self.dwg.viewbox(-self.size/2, -self.size/2, self.size, self.size)
            return

        coords = [c.center for c in elements]
        radii = [c.radius for c in elements]

        # Find the maximum extent including circle radii
        max_extent = max(max(abs(z.real) + r, abs(z.imag) + r) for z, r in zip(coords, radii))

        # Tighter padding: scale such that the max extent fits within 95% of the viewbox half-size
        self.scale_factor = (self.size / 2.1) / max_extent
        # Set the viewbox to center the drawing
        self.dwg.viewbox(-self.size/2, -self.size/2, self.size, self.size)


    def draw_scaled(self, element: Shape, **kwargs):
        """
        Draws a shape element after scaling its coordinates.

        Args:
            element: The Shape element to draw.
            **kwargs: Additional keyword arguments to pass to the element's to_svg method.
        """
        if isinstance(element, CircleElement):
            # Apply scaling to center and radius
            scaled_center = element.center * self.scale_factor
            scaled_radius = element.radius * self.scale_factor
            # Create a temporary scaled element for drawing purposes
            scaled_element = CircleElement(scaled_center, scaled_radius, element.visible)
            svg_element = scaled_element.to_svg(self.dwg, **kwargs)

        elif isinstance(element, ArcElement):
            # Scale the underlying circle and points for the arc
            scaled_circle = CircleElement(
                element.circle.center * self.scale_factor,
                element.circle.radius * self.scale_factor
            )
            # Scale start and end points
            scaled_start = element.start * self.scale_factor
            scaled_end = element.end * self.scale_factor
            # Create a temporary scaled element for drawing purposes
            scaled_element = ArcElement(
                scaled_circle,
                scaled_start,
                scaled_end,
                element.steps,
                element.visible
            )
            svg_element = scaled_element.to_svg(self.dwg, **kwargs)

        else:
            # Unknown element type; skip drawing
            svg_element = None

        if svg_element is not None:
            self.dwg.add(svg_element)

    def make_line_pattern(self, pattern_id="linePattern", spacing=10, angle=45, color="black", stroke_width=1):
        """Create a robust, angle-agnostic parallel line pattern.

        - spacing: distance between lines in user units (pixels)
        - angle: rotation in degrees (0 = vertical lines); positive is CCW
        """
        # Define a square pattern tile in user space; rotate the pattern itself
        pattern = self.dwg.pattern(
            id=pattern_id,
            patternUnits="userSpaceOnUse",
            size=(spacing, spacing),
        )

        # Rotate the pattern so the single vertical line becomes angled globally
        pattern['patternTransform'] = f"rotate({angle})"

        # Draw a vertical line spanning the tile height; duplicate at tile edge to avoid seams
        line_style = {
            'stroke': color,
            'stroke_width': stroke_width,
            'stroke_linecap': 'butt',
        }
        pattern.add(self.dwg.line(start=(0, 0), end=(0, spacing), **line_style))
        # Edge duplicate to mitigate antialiasing gaps at tile boundaries
        pattern.add(self.dwg.line(start=(spacing, 0), end=(spacing, spacing), **line_style))

        # Register pattern in defs
        self.dwg.defs.add(pattern)
        return pattern
    
    def draw_group_outline(self, points: List[complex], fill: Optional[str] = None, stroke: Optional[str] = None, stroke_width: float = 1.0, line_pattern_settings = (3, 0), use_clipped_lines: bool = False, draw_outline: bool = True):
        """
        Add a closed polygon/path for the given scaled points.

        Args:
            points: A list of complex numbers representing the scaled points of the outline.
            fill: The fill color for the polygon (if None, a polyline is drawn). Can be "pattern" or "clipped_lines".
            stroke: The stroke color for the outline.
            stroke_width: The width of the stroke.
            line_pattern_settings: Tuple of (spacing, angle) for line patterns.
            use_clipped_lines: If True, use actual clipped lines instead of pattern fills.
            draw_outline: If True, draw the polygon outline (default: True).
        """
        if not points:
            return
        # Convert complex points to (real, imag) tuples for SVG
        coords = [(p.real, p.imag) for p in points]
        
        if fill == "pattern" and not use_clipped_lines:
            # Old pattern fill method (kept for backward compatibility)
            pattern = self.make_line_pattern(pattern_id=f"angledLines_{line_pattern_settings[1]}", spacing=line_pattern_settings[0], angle=line_pattern_settings[1], color="black", stroke_width=1)
            fill_link=f"url(#{pattern['id']})"
            self.dwg.add(self.dwg.polygon(points=coords, fill=fill_link, stroke=stroke, stroke_width=stroke_width))         
        elif fill == "pattern" or fill == "clipped_lines" or use_clipped_lines:
            # New clipped lines method
            line_spacing, line_angle = line_pattern_settings
            # Get clipped line segments
            line_segments = lines_in_polygon(points, line_spacing=line_spacing, angle=line_angle, 
                                            color=stroke or "#000000", stroke_width=0.5)
            # Draw the polygon outline (only if draw_outline is True)
            if draw_outline:
                self.dwg.add(self.dwg.polygon(points=coords, fill="none", stroke=stroke, stroke_width=stroke_width))
            # Draw all the clipped lines
            for (x1, y1), (x2, y2) in line_segments:
                self.dwg.add(self.dwg.line(start=(x1, y1), end=(x2, y2), 
                                          stroke=stroke or "#000000", stroke_width=0.5))
        elif fill is not None:
            # Draw as a filled polygon
            self.dwg.add(self.dwg.polygon(points=coords, fill=fill, stroke=stroke, stroke_width=stroke_width))
        else:
            # Draw as a polyline (no fill)
            self.dwg.add(self.dwg.polyline(points=coords, fill="none", stroke=stroke or "#000000", stroke_width=stroke_width))


    def to_string(self) -> str:
        """
        Returns the SVG drawing as a string.
        """
        return self.dwg.tostring()

class CircleElement(Shape):
    """
    Represents a circle in the geometry, handling intersections.

    Each circle has a unique ID, a center (complex number), and a radius.
    It can compute intersections with other circles and find its neighbours.
    """
    _id_counter = 0

    def __init__(self, center: complex, radius: float, visible: bool = True):
        """
        Initializes a CircleElement.

        Args:
            center: The center of the circle as a complex number.
            radius: The radius of the circle.
            visible: Whether the circle should be visible in the drawing.
        """
        super().__init__(visible)
        CircleElement._id_counter += 1
        self.id = CircleElement._id_counter
        self.center = complex(center)
        self.radius = float(radius)
        # List of (point: complex, other_circle: CircleElement)
        self.intersections: List[Tuple[complex, 'CircleElement']] = []
        self.neighbours: Set['CircleElement'] = set()

    def _get_intersection_points(self, other: 'CircleElement', tol: float = 1e-6) -> List[complex]:
        """
        Calculates the intersection points between this circle and another.

        Uses the Law of Cosines to find the distance from the center to the chord connecting
        the intersection points, then finds the points on the circle along the perpendicular vector.

        Args:
            other: The other CircleElement to intersect with.
            tol: Tolerance for floating point comparisons.

        Returns:
            A list of complex numbers representing the intersection points (0, 1, or 2 points).
        """
        d = abs(self.center - other.center)
        r1, r2 = self.radius, other.radius

        # Check for no intersection, tangency (external or internal), or one circle contained within another
        if d > r1 + r2 + tol or d < abs(r1 - r2) - tol or d < tol:
            return []

        # Distance 'a' from center 1 to the chord connecting intersection points
        a = (r1**2 - r2**2 + d**2) / (2 * d)
        # Half the length of the chord, squared
        h_sq = r1**2 - a**2
        if h_sq < -tol: # Should not happen with checks above, but floating point safety
            return []
        # Half the length of the chord
        h = np.sqrt(max(h_sq, 0))

        # Midpoint of the chord
        mid = self.center + a * (other.center - self.center) / d
        # Vector perpendicular to the line between centers, scaled by 1/d
        perp_unit = 1j * (other.center - self.center) / d

        # The two intersection points
        p1, p2 = mid + h * perp_unit, mid - h * perp_unit

        # If h is near zero, the points are the same (tangency)
        return [p1] if h < tol else [p1, p2]

    def compute_intersections(self, circles: List['CircleElement'], start_reference: Optional[complex] = None, tol: float = 1e-3):
        """
        Computes and sorts intersections sequentially clockwise around the circle.
        The first point is chosen as the one closest to start_reference.

        Args:
            circles: A list of other CircleElement objects to compute intersections with.
            start_reference: A complex number used as a reference point to determine the starting point for sorting.
                             If None, the circle's center is used.
            tol: Tolerance for floating point comparisons when identifying unique points.
        """
        self.intersections.clear()
        self.neighbours.clear()
        seen = set()

        for other in circles:
            if other is self:
                continue
            # Get intersection points with another circle
            pts = self._get_intersection_points(other, tol)
            for p in pts:
                # Use a slightly coarse rounding for deduplication of points
                key = (round(p.real, 6), round(p.imag, 6))
                if key not in seen:
                    self.intersections.append((p, other))
                    seen.add(key)
                    self.neighbours.add(other)

        if not self.intersections:
            return

        c = self.center
        # Default reference point is the circle's center
        if start_reference is None:
            start_reference = c

        # Choose the starting intersection by proximity to the reference point
        start_idx = int(np.argmin([abs(p - start_reference) for p, _ in self.intersections]))
        start_point = self.intersections[start_idx][0]
        start_angle = np.angle(start_point - c)

        def clockwise_offset(angle):
            """Compute clockwise offset from start_angle in [0, 2pi)."""
            # The angle is relative to the center of *this* circle
            offset = (start_angle - angle) % (2 * np.pi)
            return offset

        # Sort intersections by clockwise offset from the starting point
        self.intersections.sort(key=lambda x: clockwise_offset(np.angle(x[0] - c)))

    def get_neighbour_circles(
        self,
        k: Optional[int] = None,
        spiral_center: complex = 0+0j,
        clockwise: bool = True,
        tie_by_distance: bool = True
    ) -> List['CircleElement']:
        """
        Return neighbours sorted by their rotation around `spiral_center` relative to this circle.

        Args:
            k: Optionally limit to the k nearest neighbours (by center distance) BEFORE angular sorting.
            spiral_center: the center of the Doyle spiral (use 0+0j in your code).
            clockwise: if True, returns neighbours in clockwise order around the spiral center
                     starting from this circle's radial direction.
            tie_by_distance: if two neighbours have nearly identical relative angle, break ties using distance.

        Returns:
            A list of neighbour CircleElement objects, sorted according to the specified criteria.
        """
        neighbours = list(self.neighbours)
        if not neighbours:
            return []

        # Optionally keep only k nearest by center distance BEFORE angular ordering:
        if k is not None and len(neighbours) > k:
            neighbours.sort(key=lambda c: abs(c.center - self.center))
            neighbours = neighbours[:k]

        # Precompute base angle of this circle around spiral center
        base_angle = np.angle(self.center - spiral_center)

        def relative_angle_to_base(other: 'CircleElement') -> float:
            # angle of neighbour around spiral center
            a = np.angle(other.center - spiral_center)
            # relative in [0, 2*pi)
            rel = (a - base_angle) % (2 * np.pi)
            return rel

        # Build sort keys: primary = relative angle, secondary = distance (optional)
        if tie_by_distance:
            neighbours.sort(key=lambda c: (relative_angle_to_base(c), abs(c.center - self.center)))
        else:
            neighbours.sort(key=relative_angle_to_base)

        # numpy.angle gives CCW angles increasing; `rel` increases CCW from base direction.
        # If user wants clockwise order, reverse the CCW ordering.
        if clockwise:
            neighbours = list(reversed(neighbours))

        return neighbours


    def to_svg(self, dwg: svgwrite.Drawing, color="#4CB39B", opacity=0.8):
        """
        Renders the circle to an SVG element.

        Args:
            dwg: The svgwrite Drawing object.
            color: The fill color of the circle.
            opacity: The fill opacity of the circle.

        Returns:
            An svgwrite Circle object, or None if the circle is not visible.
        """
        if not self.visible:
            return None
        cx, cy = self.center.real, self.center.imag
        return dwg.circle(center=(cx, cy), r=self.radius, fill=color, fill_opacity=opacity)

class ArcElement(Shape):
    """
    Represents a circular arc segment between two intersection points.

    An arc is defined by the circle it lies on and its start and end points.
    """
    def __init__(self, circle: CircleElement, start: complex, end: complex, steps: int = 40, visible: bool = True):
        """
        Initializes an ArcElement.

        Args:
            circle: The CircleElement the arc lies on.
            start: The starting point of the arc as a complex number.
            end: The ending point of the arc as a complex number.
            steps: The number of discrete points to use for rendering the arc.
            visible: Whether the arc should be visible in the drawing.
        """
        super().__init__(visible)
        self.circle = circle
        self.start = complex(start)
        self.end = complex(end)
        self.steps = steps

    def get_points(self) -> List[complex]:
        """
        Calculates the discrete points defining the arc.

        Calculates the angles of the start and end points relative to the circle's center,
        then generates a sequence of points along the arc.

        Returns:
            A list of complex numbers representing the points along the arc.
        """
        c = self.circle.center
        r = self.circle.radius
        a1 = np.angle(self.start - c)
        a2 = np.angle(self.end - c)

        # Calculate the clockwise angular difference [0, 2pi)
        delta = (a2 - a1 + 2 * np.pi) % (2 * np.pi)

        # Arc is drawn clockwise; use the smaller angular magnitude direction if needed
        if delta > np.pi:
            delta -= 2 * np.pi  # Result in [-2pi, 0] or [-pi, pi] if delta was > pi

        # Generate points along the arc using linspace for angles
        angles = np.linspace(a1, a1 + delta, self.steps)
        return [c + r * np.exp(1j * a) for a in angles]

    def to_svg(self, dwg: svgwrite.Drawing, color="#000000", width=1.2):
        """
        Renders the arc to an SVG element.

        Args:
            dwg: The svgwrite Drawing object.
            color: The stroke color of the arc.
            width: The stroke width of the arc.

        Returns:
            An svgwrite Path object representing the arc, or None if the arc is not visible.
        """
        if not self.visible:
            return None
        # Get the discrete points for the arc
        pts = self.get_points()
        if not pts:
            return None
        # Create a path string from the points
        path_data = ["M", f"{pts[0].real},{pts[0].imag}"] + [f"L{p.real},{p.imag}" for p in pts[1:]]
        return dwg.path(d=" ".join(path_data), fill="none", stroke=color, stroke_width=width)

# ============================================
# ArcGroup: group of arcs to be rendered / used as a closed outline
# ============================================

class ArcGroup:
    """
    Group of ArcElement objects.

    - An ArcElement can belong to multiple ArcGroups (we store references).
    - We can attempt to produce a closed outline from the group's arcs.
    """
    _id_counter = 0

    def __init__(self, name: Optional[str] = None):
        """
        Initializes an ArcGroup.

        Args:
            name: An optional name for the group.
        """
        ArcGroup._id_counter += 1
        self.id = ArcGroup._id_counter
        self.name = name or f"arcgroup_{self.id}"
        self.arcs: List[ArcElement] = []
        # color for debug visualization
        self.debug_fill: Optional[str] = None
        self.debug_stroke: Optional[str] = None
        # public: ring layer index within the Doyle spiral (0-based from smallest radius)
        self.ring_index: Optional[int] = None

    def add_arc(self, arc: ArcElement):
        """
        Adds an ArcElement to the group.

        Args:
            arc: The ArcElement to add.
        """
        self.arcs.append(arc)

    def extend(self, arcs: List[ArcElement]):
        """
        Adds multiple ArcElement objects to the group.

        Args:
            arcs: A list of ArcElement objects to add.
        """
        self.arcs.extend(arcs)

    def clear(self):
        """
        Removes all arcs from the group.
        """
        self.arcs.clear()

    def is_empty(self) -> bool:
        """
        Checks if the group is empty.

        Returns:
            True if the group contains no arcs, False otherwise.
        """
        return len(self.arcs) == 0

    def get_all_points(self) -> List[complex]:
        """
        Return concatenation of point sequences from all arcs (in their stored order).

        Returns:
            A list of complex numbers representing all points from all arcs in the group.
        """
        pts = []
        for arc in self.arcs:
            pts.extend(arc.get_points())
        return pts

    def _match_points(self, a: complex, b: complex, tol: float = 1e-6) -> bool:
        """
        Checks if two points are approximately the same within a given tolerance.

        Args:
            a: The first point (complex number).
            b: The second point (complex number).
            tol: The tolerance for comparison.

        Returns:
            True if the points are within the tolerance, False otherwise.
        """
        return abs(a - b) <= tol


    def get_closed_outline(self, tol: float = 1e-3) -> List[complex]:
        """Attempt to order the arcs into a closed outline (list of points).

        This is a best-effort ordering:
          - try to treat each arc as directed (start->end).
          - attempt to chain arcs by matching end to start (within tol).
          - reverse arcs if chaining requires.
          - if multiple chains exist, try to merge by endpoint proximity.
          - returns concatenated points for the found outline (closed if endpoints match).
        """
        if not self.arcs:
            return []

        # Build mutable list of arcs to place
        remaining = list(self.arcs)
        # Each entry: (arc, pts_list)
        entries = [(arc, arc.get_points()) for arc in remaining]

        # Start with the longest arc (more points) to improve chance to form outline
        entries.sort(key=lambda e: -len(e[1]))
        ordered_pts: List[complex] = entries[0][1].copy()
        used = {0}
        changed = True

        # We'll try greedily to attach other arcs to either end
        while True:
            attached_any = False
            # Iterate over remaining arcs
            for idx, (arc, pts) in enumerate(entries):
                if idx in used:
                    continue
                # possible endpoints of the existing outline and the current arc
                start_existing = ordered_pts[0]
                end_existing = ordered_pts[-1]
                start_arc = pts[0]
                end_arc = pts[-1]

                # Try attaching the arc in its original direction
                if self._match_points(end_existing, start_arc, tol):
                    # append arc as-is (excluding the matched start point)
                    ordered_pts.extend(pts[1:])
                    used.add(idx)
                    attached_any = True
                    break
                # Try attaching the arc in the reversed direction
                elif self._match_points(end_existing, end_arc, tol):
                    # append reversed arc (excluding the matched end point)
                    rev = list(reversed(pts))
                    ordered_pts.extend(rev[1:])
                    used.add(idx)
                    attached_any = True
                    break
                # Try prepending the arc in its original direction
                elif self._match_points(start_existing, end_arc, tol):
                    # prepend arc as-is (excluding the matched end point)
                    ordered_pts = pts[:-1] + ordered_pts
                    used.add(idx)
                    attached_any = True
                    break
                # Try prepending the arc in the reversed direction
                elif self._match_points(start_existing, start_arc, tol):
                    # prepend reversed arc (excluding the matched start point)
                    rev = list(reversed(pts))
                    ordered_pts = rev[:-1] + ordered_pts
                    used.add(idx)
                    attached_any = True
                    break
            # If no arcs were attached in this pass, stop trying to greedily chain
            if not attached_any:
                break

        # If some arcs were not used, attempt to attach them by nearest endpoint heuristics
        remaining_idxs = [i for i in range(len(entries)) if i not in used]
        for idx in remaining_idxs:
            pts = entries[idx][1]
            # decide where to attach by nearest endpoint
            d_front = min(abs(pts[0] - ordered_pts[0]), abs(pts[-1] - ordered_pts[0]))
            d_back = min(abs(pts[0] - ordered_pts[-1]), abs(pts[-1] - ordered_pts[-1]))
            if d_front < d_back:
                # attach to front of the existing outline
                if abs(pts[-1] - ordered_pts[0]) <= abs(pts[0] - ordered_pts[0]):
                    # Attach original arc if its end is closer to existing front
                    ordered_pts = pts[:-1] + ordered_pts
                else:
                    # Attach reversed arc if its start is closer to existing front
                    ordered_pts = list(reversed(pts))[:-1] + ordered_pts
            else:
                # attach to back of the existing outline
                if abs(pts[0] - ordered_pts[-1]) <= abs(pts[-1] - ordered_pts[-1]):
                    # Attach original arc if its start is closer to existing back
                    ordered_pts.extend(pts[1:])
                else:
                    # Attach reversed arc if its end is closer to existing back
                    ordered_pts.extend(list(reversed(pts))[1:])

        # Attempt to close: if endpoints are within tol, make sure last == first
        if ordered_pts and abs(ordered_pts[0] - ordered_pts[-1]) <= tol:
            ordered_pts[-1] = ordered_pts[0]  # explicitly set the last point to be the same as the first
        return ordered_pts

    def to_svg_fill(self, context: DrawingContext, debug: bool = False, fill_opacity: float = 0.25, pattern_fill: bool = False, line_settings = (2,0), use_clipped_lines: bool = True, draw_outline: bool = True):
        """
        Render group outline as filled polygon if closed (or as polyline) for debug.

        The context expects unscaled points; we will scale them with context.scale_factor.

        Args:
            context: The DrawingContext to use for rendering.
            debug: Whether to render with debug colors (fill and stroke).
            fill_opacity: The opacity of the fill when debug is True.
            pattern_fill: Whether to use pattern/line fill.
            line_settings: Tuple of (spacing, angle) for line patterns.
            use_clipped_lines: If True, use actual clipped lines instead of SVG patterns (default: True).
            draw_outline: If True, draw the polygon outline (default: True).
        """
        # Get the points for the outline
        pts = self.get_closed_outline()
        if not pts:
            return
        # scale points using the drawing context's scale factor
        scaled = [p * context.scale_factor for p in pts]
        if debug:
            # Generate a random color if debug colors are not set
            fill = self.debug_fill or "#%06x" % random.randint(0, 0xFFFFFF)
            stroke = self.debug_stroke or "#000000"
            # set fill and stroke and draw as a polygon
            context.draw_group_outline(scaled, fill=fill, stroke=stroke, stroke_width=0.8)
        elif pattern_fill:
            stroke = self.debug_stroke or "#000000"
            context.draw_group_outline(scaled, fill="pattern", stroke=stroke, stroke_width=0.8, 
                                      line_pattern_settings=line_settings, use_clipped_lines=use_clipped_lines,
                                      draw_outline=draw_outline)
        else:
            # Only draw outline if draw_outline is explicitly True when pattern_fill is False
            if draw_outline:
                context.draw_group_outline(scaled, fill=None, stroke="#000000", stroke_width=0.6)

# ============================================
# Doyle Spiral Class
# ============================================

class DoyleSpiral:
    """Manages the generation, intersection, and rendering of a Doyle spiral."""
    def __init__(self, p: int = 7, q: int = 32, t: float = 0, max_d: float = 2000, arc_mode: str = "closest", num_gaps: int = 2):
        """
        Initializes a DoyleSpiral.

        Args:
            p: The p parameter for the spiral.
            q: The q parameter for the spiral.
            t: The t parameter for the spiral.
            max_d: The maximum distance from the center for generating circles.
            arc_mode: The mode for selecting arcs ('closest', 'farthest', 'alternating', 'all', 'random', 'symmetric', 'angular').
            num_gaps: The number of "gaps" or arcs not to draw in 'arram_boyle' mode.
        """
        self.p, self.q, self.t, self.max_d = p, q, t, max_d
        self.arc_mode = arc_mode
        self.num_gaps = num_gaps
        # Solve the underlying Doyle system for the given parameters
        self.root = DoyleMath.solve(p, q)
        self.circles: List[CircleElement] = []
        self.outer_circles: List[CircleElement] = []
        self._is_generated = False

        # ArcGroups keyed by circle id or arbitrary name
        self.arc_groups: Dict[str, ArcGroup] = {}

    def generate_circles(self):
        """Generates the main set of visible circles based on the spiral parameters."""
        # Extract parameters from the Doyle solution
        r = self.root["r"]
        start = self.root["a"]
        scale = self.root["mod_a"] ** self.t
        alpha = self.root["arg_a"] * self.t
        min_d = 1 / scale
        a, b = self.root["a"], self.root["b"]
        w = np.exp(1j * alpha)

        circles = []
        # Generate q families of circles
        for _ in range(1, self.q + 1):
            # Generate circles moving outward from the center
            qv = start
            mod_q = abs(qv)
            while mod_q < self.max_d:
                center = scale * qv * w
                circles.append(CircleElement(center, r * scale * mod_q))
                qv *= a
                mod_q *= abs(a)

            # Generate circles moving inward towards the center
            qv = start / a # Start one step inward from the base
            mod_q = abs(qv)
            while mod_q > min_d:
                center = scale * qv * w
                circles.append(CircleElement(center, r * scale * mod_q))
                qv /= a
                mod_q /= abs(a)

            # Move to the next family of circles
            start *= b

        self.circles = circles
        self._is_generated = True

    def generate_outer_circles(self):
        """Generates exactly one outer ring of invisible circles for Arram-Boyle closure."""
        r = self.root["r"]
        start = self.root["a"]
        scale = self.root["mod_a"] ** self.t
        alpha = self.root["arg_a"] * self.t
        a, b = self.root["a"], self.root["b"]
        w = np.exp(1j * alpha)

        outer_circles = []
        # Generate one outer circle for each of the q families
        for _ in range(1, self.q + 1):
            qv = start
            # Fast-forward to the last generated visible circle's 'qv'
            while abs(qv) * scale < self.max_d:
                qv *= a

            # Add exactly one more circle (the next one outward)
            center = scale * qv * w
            # Use a generous multiplier for max_d check to ensure we get the next ring
            if abs(qv) * scale < self.max_d * abs(a) * 2:
                outer_circles.append(CircleElement(center, r * scale * abs(qv), visible=False))

            start *= b

        self.outer_circles = outer_circles

    def compute_all_intersections(self):
        """Computes all intersections for visible and outer circles."""
        all_circles = self.circles + self.outer_circles
        for c in all_circles:
            # All circles need the spiral center (0+0j) as the reference for sorting
            c.compute_intersections(all_circles, start_reference=0+0j)

    # ---- ArcGroup management APIs ----
    def create_group_for_circle(self, circle: CircleElement, name: Optional[str] = None) -> ArcGroup:
        """
        Create an ArcGroup for a specific circle.

        Args:
            circle: The CircleElement for which to create the group.
            name: An optional name for the group. If None, a name based on the circle ID is used.

        Returns:
            The created ArcGroup object.
        """
        key = name or f"circle_{circle.id}"
        group = ArcGroup(name=key)
        self.arc_groups[key] = group
        return group

    def add_arc_to_group(self, group_key: str, arc: ArcElement):
        """
        Add an ArcElement to an existing group (or create group if missing).

        Args:
            group_key: The key/name of the ArcGroup.
            arc: The ArcElement to add to the group.
        """
        if group_key not in self.arc_groups:
            self.arc_groups[group_key] = ArcGroup(name=group_key)
        self.arc_groups[group_key].add_arc(arc)

    # ---- Rendering ----

    def _render_arram_boyle(self, context: DrawingContext, debug_groups: bool = False, add_fill_pattern: bool = False, fill_pattern_spacing: float = 5.0, fill_pattern_angle: float = 0.0, red_outline: bool = False, draw_group_outline: bool = True):
        """
        Handles the Arram-Boyle rendering mode (arcs), creating ArcGroups per circle for selected arcs.

        Generates outer circles, computes intersections, selects arcs based on mode, and adds them to groups.

        Args:
            context: The DrawingContext to use for rendering.
            debug_groups: If True, render arc group outlines with debug colors.
        """
        self.generate_outer_circles()
        self.compute_all_intersections()

        all_elements = self.circles + self.outer_circles
        context.set_normalization_scale(all_elements)

        spiral_center = 0 + 0j

        # Clear existing groups (we will recreate groups for selected arcs)
        self.arc_groups.clear()

        # Precompute ring index mapping by unique radii (rounded)
        radii_all = [round(c.radius, 6) for c in self.circles]
        unique_radii = sorted(set(radii_all))
        radius_to_ring_index = {r: i for i, r in enumerate(unique_radii)}

        # 1. Draw Arcs for visible circles, but collect selected arcs into groups (one group per circle)
        for c in self.circles:
            # Only process circles with exactly 6 intersections (a key feature of this pattern)
            if len(c.intersections) == 6:
                # Select arcs to draw based on the chosen mode and number of gaps
                arcs_to_draw = ArcSelector.select_arcs_for_gaps(c, spiral_center, num_gaps=self.num_gaps, mode=self.arc_mode)
                if not arcs_to_draw:
                    continue

                # Create a group for this circle (keyed by circle id)
                group = self.create_group_for_circle(c)
                # assign ring index immediately based on this circle's radius
                group.ring_index = radius_to_ring_index.get(round(c.radius, 6), None)
                # assign debug color if requested
                if debug_groups:
                    # deterministic-ish random color per group for stability: use circle id
                    rng = random.Random(c.id)
                    color = "#%06x" % rng.randint(0, 0xFFFFFF)
                    group.debug_fill = color
                    group.debug_stroke = "#000000"

                for i, j in arcs_to_draw:
                    # Get the start and end points for the arc from the circle's sorted intersections
                    start = c.intersections[i][0]
                    end = c.intersections[j][0]
                    arc = ArcElement(c, start, end, visible=True)
                    # draw the arc line only if fill pattern is not enabled AND group outline is enabled
                    if not add_fill_pattern and draw_group_outline:
                        context.draw_scaled(arc)  # default arc color
                    # add arc to group
                    group.add_arc(arc)

        # 2. Draw Arcs from outer invisible circles (closure arcs)
        for c in self.outer_circles:
            # Outer circles should have at least 2 intersections to form an arc
            if len(c.intersections) >= 2:
                pts = [p for p, _ in c.intersections]
                n = len(pts)
                arc_distances = []
                # Calculate distances of arc midpoints to the spiral center
                for i in range(n):
                    j = (i + 1) % n
                    midpoint = (pts[i] + pts[j]) / 2
                    dist = abs(midpoint - spiral_center)
                    arc_distances.append((dist, i, j))
                # Sort arcs by distance to the spiral center
                arc_distances.sort()
                # Draw the 2nd and 3rd closest arcs (skip the closest one at index 0)
                for idx in range(1, min(3, len(arc_distances))):
                    _, i, j = arc_distances[idx]
                    start = pts[i]
                    end = pts[j]
                    arc = ArcElement(c, start, end, visible=True)
                    # Draw outer closure arcs if: red_outline is enabled OR (fill pattern disabled AND group outline enabled)
                    if red_outline or (not add_fill_pattern and draw_group_outline):
                        color = "#000000"
                        if red_outline: color = "#ff0000"
                        context.draw_scaled(arc, color=color, width=1.2)
                    # Optionally add these closure arcs to a special 'outer_closure' group
                    # so user can manually reassign them later if desired
                    key = f"outer_{c.id}"
                    if key not in self.arc_groups:
                        self.arc_groups[key] = ArcGroup(name=key)
                        # mark outer ring groups with a sentinel ring index
                        self.arc_groups[key].ring_index = -1
                        if debug_groups:
                            rng = random.Random(c.id + 1000)
                            self.arc_groups[key].debug_fill = "#%06x" % rng.randint(0, 0xFFFFFF)
                            self.arc_groups[key].debug_stroke = "#000000"
                    self.arc_groups[key].add_arc(arc)
        #"""
        # complete arc groups - This block appears to add additional arcs based on neighbor circles
        # and specific indices. This might require further review for its geometric purpose.
        max_index = max([group.ring_index for group in self.arc_groups.values()])
        for c in self.circles:
            if not f"circle_{c.id}" in self.arc_groups.keys(): continue
            group = self.arc_groups[f"circle_{c.id}"]
            neigh_lst = c.get_neighbour_circles()
            if len(neigh_lst) == 6:
                for k in [-1,-2,-5,-6]: #[0,-1,2,3]: # Indices to select neighbors
                    neigh_a = neigh_lst[k]
                    # Select all arcs from the neighbor circle
                    arcs_a = ArcSelector.select_arcs_for_gaps(neigh_a, spiral_center, mode="all")
                    #print(len(arcs_a)) # Debug print, can be removed
                    if len(arcs_a) == 6:
                        # Select specific arc index based on neighbor index k
                        arc_i = 0
                        if k == -1: arc_i = -3
                        if k == -2: arc_i = -2
                        if k == -5: arc_i = 1
                        if k == -6: arc_i = 0
                        i,j = arcs_a[arc_i]
                        # Get start and end points from the neighbor circle's intersections
                        start_a = neigh_a.intersections[i][0]
                        end_a = neigh_a.intersections[j][0]
                        # Create a new arc element from the neighbor circle
                        arc_a = ArcElement(neigh_a, start_a, end_a, visible=True)
                        # Add this arc to the current circle's group

                        group.add_arc(arc_a)
                    else:
                        # Similar logic for neighbors with a different number of arcs
                        arc_i = 0
                        if k == -1: arc_i = -3
                        if k == -2: arc_i = -2
                        if k == -5: arc_i = 1
                        if k == -6: arc_i = 0
                        i,j = arcs_a[arc_i]
                        start_a = neigh_a.intersections[i][0]
                        end_a = neigh_a.intersections[j][0]
                        arc_a = ArcElement(neigh_a, start_a, end_a, visible=True)
                        group.add_arc(arc_a)
        
        #"""
        # After drawing all arcs, render group outlines (debug fills) if debug is enabled
        if debug_groups:
            for key, group in self.arc_groups.items():
                # Exclude outer circle groups from default debug rendering
                if "outer" in key: continue
                # render group fill/outline
                group.to_svg_fill(context, debug=True, fill_opacity=0.25)

        #"""
        # After drawing all arcs, render line fillings
        if add_fill_pattern:
            for key, group in self.arc_groups.items():
                # Exclude outer circle groups from default debug rendering
                if "outer" in key: continue
                # render group fill/outline
                #group.to_svg_fill(context, debug=True, fill_opacity=0.25)
                # Interpret fill_pattern_angle as per-ring angle offset (degrees)
                ring_idx = group.ring_index if group.ring_index is not None else 0
                line_settings = (fill_pattern_spacing, ring_idx * fill_pattern_angle)
                group.to_svg_fill(context, debug=False, fill_opacity=0.25, pattern_fill=True, line_settings=line_settings, draw_outline=draw_group_outline)

        #draw red outline if option is set
        for c in self.circles:
            if not f"circle_{c.id}" in self.arc_groups.keys(): continue
            group = self.arc_groups[f"circle_{c.id}"]
            
            for i, arc in enumerate(group.arcs):
                if red_outline and (i in [3,2]) and group.ring_index == max_index: 
                    color = "#ff0000"
                    context.draw_scaled(arc, color=color, width=1.2)

        # ring_index has been assigned at creation time for inner groups and -1 for outer groups


    def _render_doyle(self, context: DrawingContext):
        """Handles the standard Doyle rendering mode (full circles)."""
        # Set normalization scale based on visible circles
        context.set_normalization_scale(self.circles)
        # Draw all visible circles
        for c in self.circles:
            context.draw_scaled(c)  # Use default circle color


    def to_svg(self, mode: str = "doyle", size: int = 800, debug_groups: bool = False, add_fill_pattern: bool = False, fill_pattern_spacing: float = 5.0, fill_pattern_angle: float = 0.0, red_outline: bool = False, draw_group_outline: bool = True) -> str:
        """
        Generates the SVG representation of the spiral in the specified mode.

        Args:
            mode: The rendering mode ('doyle' for full circles, 'arram_boyle' for arcs).
            size: The size of the output SVG (width and height).
            debug_groups: If True, render arc group outlines with debug colors in 'arram_boyle' mode.
            add_fill_pattern: If True, add line pattern fills to arc groups.
            fill_pattern_spacing: Spacing between lines in the pattern.
            fill_pattern_angle: Angle increment per ring for line patterns.
            red_outline: If True, draw red outline on specific arcs.
            draw_group_outline: If True, draw the arc group polygon outlines (default: True).

        Returns:
            A string containing the SVG representation of the spiral.

        Raises:
            ValueError: If an unknown rendering mode is provided.
        """
        # Generate circles if not already generated
        if not self._is_generated:
            self.generate_circles()

        # Create a drawing context
        context = DrawingContext(size)

        # Render based on the selected mode
        if mode == "doyle":
            self._render_doyle(context)
        elif mode == "arram_boyle":
            self._render_arram_boyle(context, debug_groups=debug_groups, add_fill_pattern=add_fill_pattern, fill_pattern_spacing=fill_pattern_spacing, fill_pattern_angle=fill_pattern_angle, red_outline=red_outline, draw_group_outline=draw_group_outline)
        else:
            raise ValueError(f"Unknown rendering mode: {mode}")

        # Return the SVG as a string
        return context.to_string()

# ============================================
# Doyle Math and Arc Selection
# ============================================

class DoyleMath:
    """Static methods for solving the Doyle spiral system."""
    @staticmethod
    def d_(z: float, t: float, p: int, q: int) -> float:
        # Helper function for the Doyle equation
        w = z ** (p / q)
        s = (p * t + 2 * np.pi) / q
        return (z * np.cos(t) - w * np.cos(s))**2 + (z * np.sin(t) - w * np.sin(s))**2

    @staticmethod
    def s_(z: float, p: int, q: int) -> float:
        # Helper function for the Doyle equation
        return (z + z ** (p / q)) ** 2

    @staticmethod
    def r_(z: float, t: float, p: int, q: int) -> float:
        # Helper function for the Doyle equation
        return DoyleMath.d_(z, t, p, q) / DoyleMath.s_(z, p, q)

    @staticmethod
    def solve(p: int, q: int) -> dict:
        """
        Solves the Doyle system for a given (p, q).

        Args:
            p: The p parameter of the Doyle spiral.
            q: The q parameter of the spiral.

        Returns:
            A dictionary containing the solution parameters 'a', 'b', 'r', 'mod_a', and 'arg_a'.
        """
        # Define the system of equations to solve
        def f_(x: np.ndarray) -> List[float]:
            z, t = x
            f1 = DoyleMath.r_(z, t, 0, 1) - DoyleMath.r_(z, t, p, q)
            f2 = DoyleMath.r_(z, t, 0, 1) - DoyleMath.r_(z ** (p / q), (p * t + 2 * np.pi) / q, 0, 1)
            return [f1, f2]

        # Use scipy's root finder to solve the system
        sol = root(f_, [2.0, 0.0], tol=1e-6)
        z, t = sol.x
        # Calculate spiral parameters from the solution
        r = np.sqrt(DoyleMath.r_(z, t, 0, 1))
        a = z * np.exp(1j * t)
        b = z ** (p / q) * np.exp(1j * (p * t + 2 * np.pi) / q)
        return {"a": a, "b": b, "r": r, "mod_a": z, "arg_a": t}

class ArcSelector:
    """Static methods for selecting which arcs to draw based on a mode."""
    @staticmethod
    def select_arcs_for_gaps(
        circle: CircleElement,
        spiral_center: complex,
        num_gaps: int = 2,
        mode: str = "closest"
    ) -> List[Tuple[int, int]]:
        """
        Selects arcs from a circle based on geometric or heuristic rules.
        Returns a list of arc index pairs (start_idx, end_idx) to be drawn.

        Args:
            circle: The CircleElement from which to select arcs.
            spiral_center: The center of the Doyle spiral (used as a reference point).
            num_gaps: The number of "gaps" or arcs *not* to draw.
            mode: The selection mode ('closest', 'farthest', 'alternating', 'all', 'random', 'symmetric', 'angular').

        Returns:
            A list of tuples, where each tuple represents the start and end index of an arc to be drawn,
            based on the sorted intersection points of the circle.
        """
        # Get intersection points from the circle
        pts = [p for p, _ in circle.intersections]
        n = len(pts)
        c = circle.center # Center of the current circle
        s = spiral_center # Center of the spiral

        if n < 2:
            return []

        # Create pairs of indices representing potential arcs between consecutive intersection points
        arcs = [(i, (i + 1) % n) for i in range(n)]
        # Calculate midpoints of these potential arcs
        midpoints = [(pts[i] + pts[j]) / 2 for i, j in arcs]

        # Select arcs based on the specified mode
        if mode in ("closest", "farthest"):
            # Calculate distances of arc midpoints to the line connecting circle center and spiral center
            line_vec = s - c
            # Handle the case where the line vector is zero (circle center is spiral center)
            if abs(line_vec) < 1e-6:
                 # In this case, all distances to the line are effectively zero.
                 # Sort by distance from the spiral center directly.
                 distances = [abs(m - s) for m in midpoints]
            else:
                 distances = [abs(np.imag(np.conj(line_vec) * (m - c))) / abs(line_vec) for m in midpoints]

            # Sort arcs based on distance, reverse if mode is 'farthest'
            sorted_arcs = [arc for _, arc in sorted(zip(distances, arcs), reverse=(mode == "farthest"))]
            # Select arcs to draw (skip the ones creating gaps)
            arcs_to_draw = sorted_arcs[num_gaps:]

        elif mode == "alternating":
            # Select arcs in an alternating pattern
            if num_gaps >= n:
                return [] # Skip all if num_gaps is greater than or equal to number of arcs
            # Determine the interval for skipping
            interval = max(1, n // (num_gaps + 1))
            arcs_to_draw = [arcs[i] for i in range(n) if (i % (interval)) != 0]

        elif mode == "all":
            # Select all arcs
            arcs_to_draw = arcs
        elif mode == "random":
            # Randomly select arcs to skip
            rng = np.random.default_rng()
            skip_idxs = rng.choice(range(n), size=min(num_gaps, n), replace=False)
            arcs_to_draw = [arc for i, arc in enumerate(arcs) if i not in skip_idxs]

        elif mode == "symmetric":
            # Select symmetric gaps around the line to the spiral center
            line_vec = s - c
             # Handle the case where the line vector is zero (circle center is spiral center)
            if abs(line_vec) < 1e-6:
                 # In this case, all angles are relative to the center, sort by angle from the x-axis
                 angles = [np.angle(m - c) for m in midpoints]
                 target_angle = 0 # Reference angle is along the positive x-axis
            else:
                angles = [np.angle(m - c) for m in midpoints]
                target_angle = np.angle(s - c)

            angular_diffs = [abs(np.angle(np.exp(1j * (a - target_angle)))) for a in angles]
            sorted_indices = np.argsort(angular_diffs)

            # Choose indices for half the gaps
            num_half_gaps = num_gaps // 2
            chosen = sorted_indices[:num_half_gaps]

            # Find symmetric indices - this needs to be relative to the circle's intersections, not angle directly
            # Find the intersection points closest to the line (which correspond to the smallest angular_diffs)
            # Then find the points roughly 180 degrees around the circle from those points.
            skip_indices = set()
            for idx in chosen:
                skip_indices.add(idx) # Add the original index
                # Find the intersection point corresponding to this midpoint arc
                midpoint_pt = midpoints[idx]
                # Find the index of the intersection point that is roughly 180 degrees opposite on the circle
                opposite_angle = np.angle(midpoint_pt - c) + np.pi
                opposite_index = -1
                min_angle_diff = float('inf')
                for i in range(n):
                    pt_angle = np.angle(pts[i] - c)
                    diff = abs(np.angle(np.exp(1j * (pt_angle - opposite_angle))))
                    if diff < min_angle_diff:
                        min_angle_diff = diff
                        opposite_index = i
                # The arc starting at the opposite_index is the symmetric one
                symmetric_arc_start_index = opposite_index
                # Find which arc index starts at symmetric_arc_start_index
                for i, (start, end) in enumerate(arcs):
                    if start == symmetric_arc_start_index:
                         # Add the arc index (not the intersection point index) to the skip list
                         skip_indices.add(i)
                         break

            # If num_gaps is odd, and the circle has an intersection point very close to the line,
            # we skip the arc that crosses the line.
            if num_gaps % 2 != 0 and abs(line_vec) > 1e-6:
                 # Find the intersection point closest to the line
                 intersection_distances = [abs(np.imag(np.conj(line_vec) * (p - c))) / abs(line_vec) for p, _ in circle.intersections]
                 closest_intersection_idx = np.argmin(intersection_distances)
                 # The arc that crosses the line is likely the one starting at or ending at this point
                 # We'll skip the arc starting at this point
                 for i, (start, end) in enumerate(arcs):
                     if start == closest_intersection_idx:
                         skip_indices.add(i)
                         break

            # Select arcs to draw
            arcs_to_draw = [arc for i, arc in enumerate(arcs) if i not in skip_indices]


        elif mode == "angular":
            # Select arcs based on angular distance from the line to the spiral center
            line_vec = s - c
             # Handle the case where the line vector is zero (circle center is spiral center)
            if abs(line_vec) < 1e-6:
                 # In this case, all angles are relative to the center, sort by angle from the x-axis
                 angles = [np.angle(m - c) for m in midpoints]
                 target_angle = 0 # Reference angle is along the positive x-axis
            else:
                angles = [np.angle(m - c) for m in midpoints]
                target_angle = np.angle(s - c)

            angular_diffs = [abs(np.angle(np.exp(1j * (a - target_angle)))) for a in angles]
            # Sort arcs by angular difference
            sorted_arcs = [arc for _, arc in sorted(zip(angular_diffs, arcs))]
            # Select arcs to draw (skip the ones creating gaps)
            arcs_to_draw = sorted_arcs[num_gaps:]

        else:
            raise ValueError(f"Unknown mode '{mode}'")

        return arcs_to_draw

# ============================================
# Interactive UI
# ============================================

def spiral_ui():
    """
    Sets up the interactive UI for the Doyle Spiral with debug toggle for ArcGroups.

    Creates sliders and dropdowns for controlling spiral parameters and rendering mode,
    and displays the generated SVG. Includes controls for manually adding arcs to groups.
    """
    # Create interactive widgets for controlling spiral parameters and rendering
    p = widgets.IntSlider(value=16, min=2, max=20, step=1, description='p')
    q = widgets.IntSlider(value=16, min=4, max=40, step=1, description='q')
    t = widgets.FloatSlider(value=0, min=0, max=1, step=0.05, description='t')
    mode = widgets.Dropdown(options=['doyle', 'arram_boyle'], value='arram_boyle', description='Mode')
    arc_mode = widgets.Dropdown(options=['closest', 'farthest', 'alternating', 'all', 'random', 'symmetric', 'angular'], value='closest', description='Arc Mode')
    num_gaps = widgets.IntSlider(value=2, min=0, max=6, step=1, description='Num Gaps')
    debug = widgets.Checkbox(value=False, description="Debug: color arc groups")
    red_outline = widgets.Checkbox(value=False, description="Red outline: color outline")
    # Pattern fill UI controls
    add_fill_pattern = widgets.Checkbox(value=False, description='Add fill pattern')
    fill_pattern_spacing = widgets.FloatSlider(value=5.0, min=0.5, max=20.0, step=0.5, description='Line spacing')
    fill_pattern_angle = widgets.FloatSlider(value=0.0, min=-90.0, max=90.0, step=1.0, description='Angle (deg)')
    draw_group_outline = widgets.Checkbox(value=False, description='Draw group outline')
    out = widgets.Output() # Output widget to display the SVG and messages

    # Hold a spiral instance so manual operations can be applied to the latest generated spiral
    spiral_holder: Dict[str, Optional[DoyleSpiral]] = {"spiral": None}

    def render(_=None):
        """
        Renders the Doyle spiral based on the current widget values and displays the SVG.

        This function is called whenever a widget value changes.
        """
        with out:
            clear_output(wait=True) # Clear previous output
            # Create a new spiral instance on change to ensure clean state
            spiral = DoyleSpiral(p.value, q.value, t.value, arc_mode=arc_mode.value, num_gaps=num_gaps.value)
            spiral_holder["spiral"] = spiral # Store the current spiral instance
            try:
                # Generate SVG based on selected mode and debug option
                svg_data = spiral.to_svg(
                    mode.value,
                    debug_groups=debug.value,
                    add_fill_pattern=add_fill_pattern.value,
                    fill_pattern_spacing=fill_pattern_spacing.value,
                    fill_pattern_angle=fill_pattern_angle.value,
                    red_outline=red_outline.value,
                    draw_group_outline=draw_group_outline.value
                )
                display(SVG(svg_data)) # Display the generated SVG
                with open("golden_spiral.svg", "w", encoding="utf-8") as f:
                    f.write(svg_data)
                # After rendering, print group info for debugging if debug is enabled
                if debug.value:
                    print(f"ArcGroups created: {list(spiral.arc_groups.keys())}")
                    for k, g in spiral.arc_groups.items():
                        print(f" - {k}: {len(g.arcs)} arcs")
            except Exception as e:
                print(f"Error generating SVG: {e}") # Print error message if SVG generation fails

    # Wire up observers / callbacks: call the render function when widget values change
    for w in [p, q, t, mode, arc_mode, num_gaps, debug, add_fill_pattern, fill_pattern_spacing, fill_pattern_angle, red_outline, draw_group_outline]:
        w.observe(render, names="value")

    # Initial render when the UI is first displayed
    render()

    # Arrange the widgets in a vertical box
    controls_top = widgets.HBox([p, q, t, mode])
    controls_arc = widgets.HBox([arc_mode, num_gaps, debug, red_outline])
    controls_fill = widgets.HBox([add_fill_pattern, fill_pattern_spacing, fill_pattern_angle, draw_group_outline])
    display(widgets.VBox([controls_top, controls_arc, controls_fill, out]))


# Run in a Jupyter cell:
spiral_ui()

## Good settings

p = 8, q = 16
mode = "angular"
n_gaps = 2

p = 16, q = 32
mode = "angular"
n_gaps = 2